2.2.1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset from the .npy file
data = np.load('data.npy', allow_pickle=True)

# Extract the labels (assuming labels are in the 4th column)
labels = data[:, 3]

# Calculate the frequency of each label using NumPy's unique function
unique_labels, label_counts = np.unique(labels, return_counts=True)

# Create a bar graph to visualize the label distribution
plt.figure(figsize=(10, 6))
plt.bar(unique_labels, label_counts, align='center', alpha=0.7)
plt.xlabel('Label Name')
plt.ylabel('Frequency')
plt.title('Label Distribution in the Dataset')
plt.xticks(rotation=45)  # Rotate the x-axis labels for better readability

# Show the plot
plt.show()


2.3.1

In [ ]:
import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
class KNNClassifier:
    def __init__(self, k=5, distance_metric='manhattan', encoder_type=None):
        self.k = k
        self.distance_metric = distance_metric
        self.encoder_type = encoder_type
        self.X_train = None
        self.y_train = None
        self.X_test = None
        self.y_test = None
    def train_val_split(self,X, y, test_size=0.2, random_state=42):
        np.random.seed(random_state)
        indices = np.random.permutation(len(X))
        split_index = int(len(X) * (1 - test_size))
        train_indices, val_indices = indices[:split_index], indices[split_index:]
        X_train, X_val = X[train_indices], X[val_indices]
        y_train, y_val = y[train_indices], y[val_indices]
        x_train = [item[0][0] for item in X_train]
        x_val = [item[0][0] for item in X_val]
            
        return x_train, x_val, y_train, y_val

    def unshuffled_train_val_split(self,X,y,test_size=0.2):
        total_samples = len(X)
        split_index = int(total_samples * (1 - test_size))

        X_train = X[:split_index]
        X_test = X[split_index:]
        y_train = y[:split_index]
        y_test = y[split_index:]
        x_train = [item[0][0] for item in X_train]
        x_val = [item[0][0] for item in X_test]
        return x_train, x_val, y_train, y_test
    def fit(self, data):
        if self.encoder_type == 'VIT':
            X_vit = data[:, 2:3]
            y = data[:, 3] 
            self.X_train, self.X_test, self.y_train, self.y_test = self.train_val_split(X_vit, y)
        elif self.encoder_type == 'Resnet':
            X_resnet = data[:, 1:2]
            y = data[:, 3] 
            self.X_train, self.X_test, self.y_train, self.y_test = self.train_val_split(X_resnet, y)

    def euclidean_distance(self, x1, x2):
        return np.linalg.norm(x1-x2,axis=1,ord=2)
    def manhattan_distance(self, x1, x2):
        return np.linalg.norm(x1-x2,axis=1,ord=1)
    def cosine_distance(self, x1, x2) -> float:
        return 1-np.dot(x2,x1)/(np.linalg.norm(x2,axis=1)*np.linalg.norm(x1))
    def calculate_distance(self, x1, x2):
        if self.distance_metric == 'euclidean':
            return self.euclidean_distance(x1, x2)
        elif self.distance_metric == 'manhattan':
            return self.manhattan_distance(x1, x2)
        elif self.distance_metric == 'cosine':
            return self.cosine_distance(x1, x2)
        else:
            raise ValueError("Unsupported distance metric")
    def fitpt2(self, data, X_test):
        if self.encoder_type == 'VIT':
            X_vit = data[:, 2:3]
            y = data[:, 3] 
            self.X_train = X_vit
            self.y_train = y
            self.X_test = X_test
        elif self.encoder_type == 'Resnet':
            X_resnet = data[:, 1:2]
            y = data[:, 3] 
            self.X_train = X_resnet
            self.y_train = y
            self.X_test = X_test
        
    def predict(self):
        y_pred = []
        for x in self.X_test:
            distances = self.calculate_distance(x, self.X_train)
            sorted_indices = np.argsort(distances)
            k_indices = sorted_indices[:self.k]
            k_nearest_labels = [self.y_train[i] for i in k_indices]
            unique_labels, counts = np.unique(k_nearest_labels, return_counts=True)
            pred_label = unique_labels[np.argmax(counts)]
            y_pred.append(pred_label)
        print(self.encoder_type)
        print("Accuracy:" + str(accuracy_score(self.y_test,y_pred)))
        print("F1 Score:" + str(f1_score(self.y_test,y_pred,average='macro')))
        print("Precision score:" + str(precision_score(self.y_test,y_pred,average='macro',zero_division=0)))
        print("Recall score:" + str(recall_score(self.y_test,y_pred,average='macro',zero_division=0)))
def train_val_split(X, y, test_size=0.2, random_state=42):
    np.random.seed(random_state)
    indices = np.random.permutation(len(X))
    split_index = int(len(X) * (1 - test_size))
    train_indices, val_indices = indices[:split_index], indices[split_index:]
    X_train, X_val = X[train_indices], X[val_indices]
    y_train, y_val = y[train_indices], y[val_indices]
    x_train = [item[0][0] for item in X_train]
    x_val = [item[0][0] for item in X_val]
        
    return x_train, x_val, y_train, y_val
data = np.load('data.npy', allow_pickle=True)
X_vit = data[:, 2:3]
y = data[:, 3] 
X_train, X_test, y_train, y_test = train_val_split(X_vit, y)
knn_classifier = KNeighborsClassifier()
start_time = time.time()
vit_knn = KNNClassifier(k=3, distance_metric='euclidean',encoder_type='VIT')
vit_knn.fit(data=data)
vit_knn.predict()
end_time = time.time()
knn_classifier = KNeighborsClassifier(n_neighbors=3, metric='euclidean')
knn_classifier.fit(X_train, y_train)
y_pred = knn_classifier.predict(X_test)
print(accuracy_score(y_test, y_pred))
end2_time = time.time()
resnet_knn = KNNClassifier(k=5, distance_metric='cosine',encoder_type='Resnet')
resnet_knn.fit(data=data)
resnet_knn.predict()
execution_time = end_time - start_time
execution_time2 = end2_time - end_time
print(f"Execution time1: {execution_time} seconds")
print(f"Execution time2: {execution_time2} seconds")



2.4.1.1 & 2.4.1.2

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
class KNNClassifier:
    def __init__(self, k=5, distance_metric='manhattan', encoder_type=None):
        self.k = k
        self.distance_metric = distance_metric
        self.encoder_type = encoder_type
        self.X_train = None
        self.y_train = None
        self.X_test = None
        self.y_test = None
    def train_val_split(self,X, y, test_size=0.2, random_state=42):
        np.random.seed(random_state)
        indices = np.random.permutation(len(X))
        split_index = int(len(X) * (1 - test_size))
        train_indices, val_indices = indices[:split_index], indices[split_index:]
        X_train, X_val = X[train_indices], X[val_indices]
        y_train, y_val = y[train_indices], y[val_indices]
        x_train = [item[0][0] for item in X_train]
        x_val = [item[0][0] for item in X_val]
            
        return x_train, x_val, y_train, y_val

    def unshuffled_train_val_split(self,X,y,test_size=0.2):
        total_samples = len(X)
        split_index = int(total_samples * (1 - test_size))

        X_train = X[:split_index]
        X_test = X[split_index:]
        y_train = y[:split_index]
        y_test = y[split_index:]
        x_train = [item[0][0] for item in X_train]
        x_val = [item[0][0] for item in X_test]
        return x_train, x_val, y_train, y_test
    def fit(self, data):
        if self.encoder_type == 'VIT':
            X_vit = data[:, 2:3]
            y = data[:, 3] 
            self.X_train, self.X_test, self.y_train, self.y_test = self.train_val_split(X_vit, y)
        elif self.encoder_type == 'Resnet':
            X_resnet = data[:, 1:2]
            y = data[:, 3] 
            self.X_train, self.X_test, self.y_train, self.y_test = self.train_val_split(X_resnet, y)

    def euclidean_distance(self, x1, x2):
        # return np.sqrt(np.sum((x1 - x2) ** 2),axis=1)
        return np.linalg.norm(x1-x2,axis=1,ord=2)

    def manhattan_distance(self, x1, x2):
        return np.linalg.norm(x1-x2,axis=1,ord=1)

    def cosine_distance(self, x1, x2):
        return 1-np.dot(x2,x1)/(np.linalg.norm(x2,axis=1)*np.linalg.norm(x1))

    def calculate_distance(self, x1, x2):
        if self.distance_metric == 'euclidean':
            return self.euclidean_distance(x1, x2)
        elif self.distance_metric == 'manhattan':
            return self.manhattan_distance(x1, x2)
        elif self.distance_metric == 'cosine':
            return self.cosine_distance(x1, x2)
        else:
            raise ValueError("Unsupported distance metric")
    def fitpt2(self, data, X_test):
        if self.encoder_type == 'VIT':
            X_vit = data[:, 2:3]
            y = data[:, 3] 
            self.X_train = X_vit
            self.y_train = y
            self.X_test = X_test
        elif self.encoder_type == 'Resnet':
            X_resnet = data[:, 1:2]
            y = data[:, 3] 
            self.X_train = X_resnet
            self.y_train = y
            self.X_test = X_test
        
    def predict(self):
        y_pred = []
        for x in self.X_test:
            distances = self.calculate_distance(x, self.X_train)
            sorted_indices = np.argsort(distances)
            k_indices = sorted_indices[:self.k]
            k_nearest_labels = [self.y_train[i] for i in k_indices]
            unique_labels, counts = np.unique(k_nearest_labels, return_counts=True)
            pred_label = unique_labels[np.argmax(counts)]
            y_pred.append(pred_label)
        return accuracy_score(self.y_test,y_pred)
    
data = np.load('data.npy', allow_pickle=True)
k_value = []
accuracy = []
answer = []
for i in ['VIT', 'Resnet']:
    for j in ['manhattan', 'euclidean', 'cosine']:
        for kk in range(1,10,2):
            knn = KNNClassifier(k=kk,distance_metric=j,encoder_type=i)
            knn.fit(data=data)
            ans = knn.predict()
            answer.append([ans,kk,j,i])
            k_value.append(i)
            accuracy.append(ans)
answer.sort(reverse=True)
print("The best triplet is: ")
print("K = " + str(answer[0][1]))
print("Distance metric = " + str(answer[0][2]))
print("Encoder type: " + str(answer[0][3]))
j=0
print()
print("The top 20 triplets are: ")
for i in answer:
    print(i)
    j += 1
    if(j == 20):
        break

2.4.1.3

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
class KNNClassifier:
    def __init__(self, k=5, distance_metric='manhattan', encoder_type=None):
        self.k = k
        self.distance_metric = distance_metric
        self.encoder_type = encoder_type
        self.X_train = None
        self.y_train = None
        self.X_test = None
        self.y_test = None
    def train_val_split(self,X, y, test_size=0.2, random_state=42):
        np.random.seed(random_state)
        indices = np.random.permutation(len(X))
        split_index = int(len(X) * (1 - test_size))
        train_indices, val_indices = indices[:split_index], indices[split_index:]
        X_train, X_val = X[train_indices], X[val_indices]
        y_train, y_val = y[train_indices], y[val_indices]
        x_train = [item[0][0] for item in X_train]
        x_val = [item[0][0] for item in X_val]
            
        return x_train, x_val, y_train, y_val

    def unshuffled_train_val_split(self,X,y,test_size=0.2):
        total_samples = len(X)
        split_index = int(total_samples * (1 - test_size))

        X_train = X[:split_index]
        X_test = X[split_index:]
        y_train = y[:split_index]
        y_test = y[split_index:]
        x_train = [item[0][0] for item in X_train]
        x_val = [item[0][0] for item in X_test]
        return x_train, x_val, y_train, y_test
    def fit(self, data):
        if self.encoder_type == 'VIT':
            X_vit = data[:, 2:3]
            y = data[:, 3] 
            self.X_train, self.X_test, self.y_train, self.y_test = self.train_val_split(X_vit, y)
        elif self.encoder_type == 'Resnet':
            X_resnet = data[:, 1:2]
            y = data[:, 3] 
            self.X_train, self.X_test, self.y_train, self.y_test = self.train_val_split(X_resnet, y)

    def euclidean_distance(self, x1, x2):
        # return np.sqrt(np.sum((x1 - x2) ** 2),axis=1)
        return np.linalg.norm(x1-x2,axis=1,ord=2)

    def manhattan_distance(self, x1, x2):
        return np.linalg.norm(x1-x2,axis=1,ord=1)

    def cosine_distance(self, x1, x2):
        return 1-np.dot(x2,x1)/(np.linalg.norm(x2,axis=1)*np.linalg.norm(x1))

    def calculate_distance(self, x1, x2):
        if self.distance_metric == 'euclidean':
            return self.euclidean_distance(x1, x2)
        elif self.distance_metric == 'manhattan':
            return self.manhattan_distance(x1, x2)
        elif self.distance_metric == 'cosine':
            return self.cosine_distance(x1, x2)
        else:
            raise ValueError("Unsupported distance metric")
    def fitpt2(self, data, X_test):
        if self.encoder_type == 'VIT':
            X_vit = data[:, 2:3]
            y = data[:, 3] 
            self.X_train = X_vit
            self.y_train = y
            self.X_test = X_test
        elif self.encoder_type == 'Resnet':
            X_resnet = data[:, 1:2]
            y = data[:, 3] 
            self.X_train = X_resnet
            self.y_train = y
            self.X_test = X_test
        
    def predict(self):
        y_pred = []
        #print(len(X_test[0][0]))
        #print(X_test)
        for x in self.X_test:
            #print(x)
            #distances = [self.calculate_distance(x[0][0], x_train[0][0]) for x_train in self.X_train]
            #self.X_train = self.X_train[0]
            # print(len(self.X_train))
            # print(len(x[0][0]))
            distances = self.calculate_distance(x, self.X_train)
            # distances = np.array([self.calculate_distance(x[0][0], x_train[0][0]) for x_train in self.X_train])
            sorted_indices = np.argsort(distances)
            k_indices = sorted_indices[:self.k]
            k_nearest_labels = [self.y_train[i] for i in k_indices]
            unique_labels, counts = np.unique(k_nearest_labels, return_counts=True)
            pred_label = unique_labels[np.argmax(counts)]
            y_pred.append(pred_label)
        return accuracy_score(self.y_test,y_pred)
        #return np.array(y_pred)

# Load the dataset from data.npy
data = np.load('data.npy', allow_pickle=True)
# X_resnet = data[:, 1:2] 
# X_vit = data[:, 2:3]
# y = data[:, 3] 
    

# X_resnet_train, X_resnet_test, y_resnet_train, y_resnet_test = train_val_split(X_resnet, y)
# X_vit_train, X_vit_test, y_vit_train, y_vit_test = train_val_split(X_vit, y)
k_value = []
accuracy = []
for i in range(1,10,2):
    knn = KNNClassifier(k=i,distance_metric='euclidean',encoder_type='VIT')
    knn.fit(data=data)
    ans = knn.predict()
    k_value.append(i)
    accuracy.append(ans)

print(k_value)
print(accuracy)
plt.plot(k_value, accuracy, marker='o', linestyle='-')
plt.xlabel('K')
plt.ylabel('Accuracy')
plt.title('K vs Accuracy graph')
plt.show()

2.6.1.1

2.6.1.2

2.6.1.3

3.3.1 (MultiOutput)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score

class MultiOutputDecisionTreeClassifier:
    def __init__(self, max_depth=5, max_features='auto', criterion='gini'):
        self.max_depth = max_depth
        self.max_features = max_features
        self.criterion = criterion
        self.classifier = DecisionTreeClassifier(criterion=criterion,random_state=42)
    def fit(self, X, y):
        mlb = MultiLabelBinarizer()
        y = mlb.fit_transform(y.str.split(' '))
        self.classifier.fit(X, y)
    def predict(self, X):
        return self.classifier.predict(X)
data = pd.read_csv('advertisement.csv')  # Replace with your CSV file path
print("Original DataFrame:")
print(data.head())
data.fillna(method='ffill', inplace=True) 
categorical_cols = ['gender', 'education', 'married', 'city', 'occupation', 'most bought item']
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
print("Encoded DataFrame:")
print(data_encoded.head())
X = data_encoded.drop('labels', axis=1)  # Features
y = data_encoded['labels']  # Target variable
print(y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(y_train)
mlb = MultiLabelBinarizer()
# y_train = y_train.str.get_dummies(sep=' ')
# y_train = mlb.fit_transform(y_train)
y_val = mlb.fit_transform(y_val.str.split(' '))
# y_val = y_val.str.get_dummies(sep=' ')
# print(y_train)
print(y_val)
clf = MultiOutputDecisionTreeClassifier(max_depth=5, max_features = 5,criterion='gini')
clf.fit(X_train, y_train)
val_predictions = clf.predict(X_val)
print(val_predictions)
accuracy = accuracy_score(y_val, val_predictions)
micro_f1 = f1_score(y_val, val_predictions, average='micro')
macro_f1 = f1_score(y_val, val_predictions, average='macro')
conf_matrix = confusion_matrix(y_val.argmax(axis=1), val_predictions.argmax(axis=1))
precision = precision_score(y_val, val_predictions, average='micro')
recall = recall_score(y_val, val_predictions, average='micro')

print(f'Accuracy: {accuracy:.2f}')
print(f'F1 (Micro): {micro_f1:.2f}')
print(f'F1 (Macro): {macro_f1:.2f}')
print('Confusion Matrix:')
print(conf_matrix)
print(f'Precision (Micro): {precision:.2f}')
print(f'Recall (Micro): {recall:.2f}')
# val_predictions = mlb.inverse_transform(val_predictions)
# y_val = mlb.inverse_transform(y_val)

# Initialize an empty dictionary to store confusion matrices for each label
# confusion_matrices = {}

# # Compute confusion matrix for each label
# for label_idx, label in enumerate(mlb.classes_):
#     y_val_label = [1 if label in labels else 0 for labels in y_val]
#     val_predictions_label = [1 if label in labels else 0 for labels in val_predictions]
#     cm = confusion_matrix(y_val_label, val_predictions_label)
#     confusion_matrices[label] = cm

# # Print confusion matrices for each label
# for label, cm in confusion_matrices.items():
#     print(f'Confusion Matrix for Label "{label}":')
#     print(cm)


3.3.1 (Powerset)

In [ ]:
import numpy as np
import pandas as pd
from itertools import chain, combinations
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score

class PowersetDecisionTreeClassifier:
    def __init__(self, max_depth=5, max_features='auto', criterion='gini'):
        self.max_depth = max_depth
        self.max_features = max_features
        self.criterion = criterion
        self.classifier = DecisionTreeClassifier(max_depth=max_depth,max_features=max_features,criterion=criterion,random_state=42)
    def fit(self, X, y):
        self.classifier.fit(X, y)
    def predict(self, X):
        return self.classifier.predict(X)
data = pd.read_csv('advertisement.csv')  # Replace with your CSV file path
# print("Original DataFrame:")
# print(data.head())
data.fillna(method='ffill', inplace=True) 
categorical_cols = ['gender', 'education', 'married', 'city', 'occupation', 'most bought item']
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
# print("Encoded DataFrame:")
# print(data_encoded.head())
X = data_encoded.drop('labels', axis=1)  # Features
y = data_encoded['labels']  # Target variable
# print(y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# print(y_train)
def powerset(iterable):
    s = list(iterable)
    unique_combos = set()
    for r in range(0,len(s) + 1):
        for combo in combinations(sorted(s), r):
            # return tuple(combo)
            unique_combos.add(tuple(combo))
    return list(unique_combos)
label_combinations = y_train.apply(lambda x: tuple(x.split(' '))).apply(powerset)
label_combinations_val = y_val.apply(lambda x: tuple(x.split(' '))).apply(powerset)
print(label_combinations)
unique_labels = y.str.split(' ').explode().unique()  
all_labels = list(chain.from_iterable(combinations(unique_labels, r) for r in range(len(unique_labels) + 1)))
all_label_combinations = all_labels
#print(label_combinations_val)
# all_label_combinations = list(set(label_combinations.sum() + label_combinations_val.sum()))
# print(all_label_combinations)
print(label_combinations_val)
label_combinations_list = label_combinations.tolist()
label_combinations_val_list = label_combinations_val.tolist()
print(label_combinations_val_list)
#print(label_combinations_list)
mlb = MultiLabelBinarizer(classes=all_label_combinations)
y_train = mlb.fit_transform(label_combinations_list)
y_train_df = pd.DataFrame(y_train, columns=mlb.classes_)
y_train = y_train_df.loc[:, ~y_train_df.columns.duplicated()]
y_val = mlb.fit_transform(label_combinations_val_list)
y_val_df = pd.DataFrame(y_val, columns=mlb.classes_)
# # Remove duplicate columns, if any
y_val = y_val_df.loc[:, ~y_val_df.columns.duplicated()]
# # y_val = mlb.fit_transform(y_val.apply(lambda x: tuple(x.split(' '))))
#print(y_train)
print(y_val)
clf = PowersetDecisionTreeClassifier(max_depth=30, max_features = 11,criterion='entropy')
clf.fit(X_train, y_train)
val_predictions = clf.predict(X_val)
# print(val_predictions)
y_val = y_val.to_numpy()
print(np.sum(y_val[0]))
accuracy = accuracy_score(y_val, val_predictions)
micro_f1 = f1_score(y_val, val_predictions, average='micro')
macro_f1 = f1_score(y_val, val_predictions, average='macro')
conf_matrix = confusion_matrix(y_val.argmax(axis=1), val_predictions.argmax(axis=1))
precision = precision_score(y_val, val_predictions, average='micro')
recall = recall_score(y_val, val_predictions, average='micro')

print(f'Accuracy: {accuracy:.5f}')
print(f'F1 (Micro): {micro_f1:.5f}')
print(f'F1 (Macro): {macro_f1:.5f}')
print('Confusion Matrix:')
print(conf_matrix)
print(f'Precision (Micro): {precision:.5f}')
print(f'Recall (Micro): {recall:.5f}')
# val_predictions = mlb.inverse_transform(val_predictions)
# y_val = mlb.inverse_transform(y_val)

# Initialize an empty dictionary to store confusion matrices for each label
# confusion_matrices = {}

# # Compute confusion matrix for each label
# for label_idx, label in enumerate(mlb.classes_):
#     y_val_label = [1 if label in labels else 0 for labels in y_val]
#     val_predictions_label = [1 if label in labels else 0 for labels in val_predictions]
#     cm = confusion_matrix(y_val_label, val_predictions_label)
#     confusion_matrices[label] = cm

# # Print confusion matrices for each label
# for label, cm in confusion_matrices.items():
#     print(f'Confusion Matrix for Label "{label}":')
#     print(cm)


3.4.1.1 (MultiOutput)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score

class MultiOutputDecisionTreeClassifier:
    def __init__(self, max_depth=5, max_features='auto', criterion='gini'):
        self.max_depth = max_depth
        self.max_features = max_features
        self.criterion = criterion
        self.classifier = DecisionTreeClassifier(max_depth=max_depth,max_features=max_features,criterion=criterion,random_state=42)
    def fit(self, X, y):
        self.classifier.fit(X, y)
    def predict(self, X):
        return self.classifier.predict(X)
data = pd.read_csv('advertisement.csv')  # Replace with your CSV file path
print("Original DataFrame:")
print(data.head())
data.fillna(method='ffill', inplace=True) 
categorical_cols = ['gender', 'education', 'married', 'city', 'occupation', 'most bought item']
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
print("Encoded DataFrame:")
print(data_encoded.head())
X = data_encoded.drop('labels', axis=1)  # Features
y = data_encoded['labels']  # Target variable
print(y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(y_train)
mlb = MultiLabelBinarizer()
# y_train = y_train.str.get_dummies(sep=' ')
y_train = mlb.fit_transform(y_train.str.split(' '))
y_val = mlb.fit_transform(y_val.str.split(' '))
# y_val = y_val.str.get_dummies(sep=' ')
# print(y_train)
# print(y_val)
for i in [3,5,10,20,30]:
    for j in [3,5,7,9,11]:
        for k in ['gini','entropy']:            
            clf = MultiOutputDecisionTreeClassifier(max_depth=i, max_features = j,criterion=k)
            clf.fit(X_train, y_train)
            val_predictions = clf.predict(X_val)
            # print(val_predictions)
            accuracy = accuracy_score(y_val, val_predictions)
            micro_f1 = f1_score(y_val, val_predictions, average='micro',zero_division=0)
            macro_f1 = f1_score(y_val, val_predictions, average='macro',zero_division=0)
            conf_matrix = confusion_matrix(y_val.argmax(axis=1), val_predictions.argmax(axis=1))
            precision = precision_score(y_val, val_predictions, average='micro')
            recall = recall_score(y_val, val_predictions, average='micro')
            print("Max Depth: " + str(i) + " Max Features: " + str(j) + " Criterion: " + str(k))
            print(f'Accuracy: {accuracy:.2f}')
            print(f'F1 (Micro): {micro_f1:.2f}')
            print(f'F1 (Macro): {macro_f1:.2f}')
            print('Confusion Matrix:')
            print(conf_matrix)
            print(f'Precision (Micro): {precision:.2f}')
            print(f'Recall (Micro): {recall:.2f}')
# val_predictions = mlb.inverse_transform(val_predictions)
# y_val = mlb.inverse_transform(y_val)

# Initialize an empty dictionary to store confusion matrices for each label
# confusion_matrices = {}

# # Compute confusion matrix for each label
# for label_idx, label in enumerate(mlb.classes_):
#     y_val_label = [1 if label in labels else 0 for labels in y_val]
#     val_predictions_label = [1 if label in labels else 0 for labels in val_predictions]
#     cm = confusion_matrix(y_val_label, val_predictions_label)
#     confusion_matrices[label] = cm

# # Print confusion matrices for each label
# for label, cm in confusion_matrices.items():
#     print(f'Confusion Matrix for Label "{label}":')
#     print(cm)


3.4.1.1 (Powerset)

In [ ]:
import numpy as np
import pandas as pd
from itertools import chain, combinations
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score

class PowersetDecisionTreeClassifier:
    def __init__(self, max_depth=5, max_features='auto', criterion='gini'):
        self.max_depth = max_depth
        self.max_features = max_features
        self.criterion = criterion
        self.classifier = DecisionTreeClassifier(max_depth = max_depth, max_features=max_features,criterion=criterion,random_state=42)
    def fit(self, X, y):
        self.classifier.fit(X, y)
    def predict(self, X):
        return self.classifier.predict(X)
data = pd.read_csv('advertisement.csv')  # Replace with your CSV file path
# print("Original DataFrame:")
# print(data.head())
data.fillna(method='ffill', inplace=True) 
categorical_cols = ['gender', 'education', 'married', 'city', 'occupation', 'most bought item']
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
# print("Encoded DataFrame:")
# print(data_encoded.head())
X = data_encoded.drop('labels', axis=1)  # Features
y = data_encoded['labels']  # Target variable
# print(y)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# print(y_train)
def powerset(iterable):
    s = list(iterable)
    unique_combos = set()
    for r in range(len(s),len(s) + 1):
        for combo in combinations(sorted(s), r):
            return tuple(combo)
            unique_combos.add(tuple(combo))
    return list(unique_combos)
label_combinations = y_train.apply(lambda x: tuple(x.split(' '))).apply(powerset)
label_combinations_val = y_val.apply(lambda x: tuple(x.split(' '))).apply(powerset)
# print(label_combinations)

all_label_combinations = list(set(label_combinations.sum() + label_combinations_val.sum()))
unique_labels = y.str.split(' ').explode().unique()  
all_label_combinations = list(chain.from_iterable(combinations(unique_labels, r) for r in range(len(unique_labels) + 1)))
label_combinations_list = label_combinations.tolist()
label_combinations_val_list = label_combinations_val.tolist()
#print(label_combinations_list)
print(label_combinations_val_list)
all_label_combinations = sorted(all_label_combinations, key=lambda x: tuple(sorted(x)))
mlb = MultiLabelBinarizer(classes=np.arange(len(all_label_combinations)))
mlb.fit(all_label_combinations)
def custom_binarize(label_combination):
    binarized = np.zeros(len(all_label_combinations))
    for idx, label_set in enumerate(all_label_combinations):
        if set(label_set) == set(label_combination):
            binarized[idx] = 1
    return binarized

# Binarize y_train and y_val using the custom binarization function
print(y_val)
y_train = np.array([custom_binarize(labels) for labels in label_combinations_list])
y_val = np.array([custom_binarize(labels) for labels in label_combinations_val_list])
# y_train = mlb.fit_transform(label_combinations_list)
# y_train_df = pd.DataFrame(y_train, columns=mlb.classes_)
# y_train = y_train_df.loc[:, ~y_train_df.columns.duplicated()]
# y_val = mlb.fit_transform(label_combinations_val_list)
# y_val_df = pd.DataFrame(y_val, columns=mlb.classes_)
# # # Remove duplicate columns, if any
# y_val = y_val_df.loc[:, ~y_val_df.columns.duplicated()]
# # y_val = mlb.fit_transform(y_val.apply(lambda x: tuple(x.split(' '))))
#print(y_train)
# print(y_val)
# print(np.sum(y_val[1]))
for i in [3,5,10,20,30]:
    for j in [3,5,7,9,11]:
            for k in ['gini','entropy']:  
                clf = PowersetDecisionTreeClassifier(max_depth=i, max_features = j,criterion=k)
                clf.fit(X_train, y_train)
                val_predictions = clf.predict(X_val)
                # print(val_predictions)
                # y_val = y_val.to_numpy()
                # print(np.sum(y_val[0]))
                # print(val_predictions)
                accuracy = accuracy_score(y_val, val_predictions)
                micro_f1 = f1_score(y_val, val_predictions, average='micro',zero_division=0)
                macro_f1 = f1_score(y_val, val_predictions, average='macro',zero_division=0)
                conf_matrix = confusion_matrix(y_val.argmax(axis=1), val_predictions.argmax(axis=1))
                precision = precision_score(y_val, val_predictions, average='micro',zero_division=0)
                recall = recall_score(y_val, val_predictions, average='micro',zero_division=0)
                print("Max Depth: " + str(i) + " Max Features: " + str(j) + " Criterion: " + str(k))
                print(f'Accuracy: {accuracy:.5f}')
                print(f'F1 (Micro): {micro_f1:.5f}')
                print(f'F1 (Macro): {macro_f1:.5f}')
                print('Confusion Matrix:')
                print(conf_matrix)
                print(f'Precision (Micro): {precision:.5f}')
                print(f'Recall (Micro): {recall:.5f}')
# val_predictions = mlb.inverse_transform(val_predictions)
# y_val = mlb.inverse_transform(y_val)

# Initialize an empty dictionary to store confusion matrices for each label
# confusion_matrices = {}

# # Compute confusion matrix for each label
# for label_idx, label in enumerate(mlb.classes_):
#     y_val_label = [1 if label in labels else 0 for labels in y_val]
#     val_predictions_label = [1 if label in labels else 0 for labels in val_predictions]
#     cm = confusion_matrix(y_val_label, val_predictions_label)
#     confusion_matrices[label] = cm

# # Print confusion matrices for each label
# for label, cm in confusion_matrices.items():
#     print(f'Confusion Matrix for Label "{label}":')
#     print(cm)

3.4.1.2 (MultiOutput)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score

class MultiOutputDecisionTreeClassifier:
    def __init__(self, max_depth, max_features, criterion):
        self.max_depth = max_depth
        self.max_features = max_features
        self.criterion = criterion
        self.classifier = DecisionTreeClassifier(max_depth=max_depth, max_features = max_features,criterion=criterion,random_state=42)
    def fit(self, X, y):
        self.classifier.fit(X, y)
    def predict(self, X):
        return self.classifier.predict(X)
data = pd.read_csv('advertisement.csv') 
data.fillna(method='ffill', inplace=True) 
categorical_cols = ['gender', 'education', 'married', 'city', 'occupation', 'most bought item']
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
X = data_encoded.drop('labels', axis=1) 
y = data_encoded['labels'] 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
# test_size = 0.2

# # Calculate the number of data points to reserve for testing
# num_test_samples = int(len(X) * test_size)

# # Create the training and testing sets without shuffling
# X_train, X_val = X[:num_test_samples], X[num_test_samples:]
# y_train, y_val = y[:num_test_samples], y[num_test_samples:]
mlb = MultiLabelBinarizer()
# y_train = y_train.str.get_dummies(sep=' ')
y_train = mlb.fit_transform(y_train.str.split(' '))
y_val = mlb.fit_transform(y_val.str.split(' '))
print(y_train[1])
ans = []
for i in [3,5,10,20,30]:
    for j in [3,5,7,9,11]:
        for k in ['gini','entropy']:
            clf = MultiOutputDecisionTreeClassifier(max_depth=i, max_features = j,criterion=k)
            clf.fit(X_train, y_train)
            val_predictions = clf.predict(X_val)
            # print(y_val)
            macro_f1 = f1_score(y_val, val_predictions, average='macro')
            ans.append([macro_f1,i,j,k])
ans.sort(reverse=True)
for i in range(0,3):
    print("F1 Score: " + str(ans[i][0]) + " Max Depth: " + str(ans[i][1]) + " Max Features: " + str(ans[i][2]) + " Criterion: " + str(ans[i][3]))


3.4.1.2 (Powerset)

In [ ]:
import numpy as np
import pandas as pd
from itertools import chain, combinations
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score

class PowersetDecisionTreeClassifier:
    def __init__(self, max_depth=5, max_features='auto', criterion='gini'):
        self.max_depth = max_depth
        self.max_features = max_features
        self.criterion = criterion
        self.classifier = DecisionTreeClassifier(max_depth = max_depth, max_features=max_features,criterion=criterion,random_state=42)
    def fit(self, X, y):
        self.classifier.fit(X, y)
    def predict(self, X):
        return self.classifier.predict(X)
data = pd.read_csv('advertisement.csv') 
data.fillna(method='ffill', inplace=True) 
categorical_cols = ['gender', 'education', 'married', 'city', 'occupation', 'most bought item']
data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
X = data_encoded.drop('labels', axis=1)  
y = data_encoded['labels']  
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
def powerset(iterable):
    s = list(iterable)
    unique_combos = set()
    for r in range(len(s),len(s) + 1):
        for combo in combinations(sorted(s), r):
            return tuple(combo)
            unique_combos.add(tuple(combo))
    return list(unique_combos)
label_combinations = y_train.apply(lambda x: tuple(x.split(' '))).apply(powerset)
label_combinations_val = y_val.apply(lambda x: tuple(x.split(' '))).apply(powerset)
all_label_combinations = list(set(label_combinations.sum() + label_combinations_val.sum()))
unique_labels = y.str.split(' ').explode().unique()  
all_label_combinations = list(chain.from_iterable(combinations(unique_labels, r) for r in range(len(unique_labels) + 1)))
label_combinations_list = label_combinations.tolist()
label_combinations_val_list = label_combinations_val.tolist()
all_label_combinations = sorted(all_label_combinations, key=lambda x: tuple(sorted(x)))
mlb = MultiLabelBinarizer(classes=np.arange(len(all_label_combinations)))
mlb.fit(all_label_combinations)
def custom_binarize(label_combination):
    binarized = np.zeros(len(all_label_combinations))
    for idx, label_set in enumerate(all_label_combinations):
        if set(label_set) == set(label_combination):
            binarized[idx] = 1
    return binarized

# Binarize y_train and y_val using the custom binarization function
print(label_combinations_list[0])
print(label_combinations_val_list[0])
y_train = np.array([custom_binarize(labels) for labels in label_combinations_list])
y_val = np.array([custom_binarize(labels) for labels in label_combinations_val_list])
# print(np.sum(y_train[0])
# )
# y_train = mlb.fit_transform(label_combinations_list)
# y_train_df = pd.DataFrame(y_train, columns=mlb.classes_)
# y_train = y_train_df.loc[:, ~y_train_df.columns.duplicated()]
# y_val = mlb.fit_transform(label_combinations_val_list)
# y_val_df = pd.DataFrame(y_val, columns=mlb.classes_)
# # # Remove duplicate columns, if any
# y_val = y_val_df.loc[:, ~y_val_df.columns.duplicated()]
# # y_val = mlb.fit_transform(y_val.apply(lambda x: tuple(x.split(' '))))
#print(y_train)
# print(y_val)
# print(np.sum(y_val[1]))
ans = []
for i in [3,5,10,20,30]:
    for j in [3,5,7,9,11]:
            for k in ['gini','entropy']:  
                clf = PowersetDecisionTreeClassifier(max_depth=i, max_features = j,criterion=k)
                clf.fit(X_train, y_train)
                val_predictions = clf.predict(X_val)
                # sum = 0
                # for i in range(len(val_predictions)):
                #     sum = sum + np.sum(val_predictions[i])
                # print(sum)
                # y_val = y_val.to_numpy()
                # print(np.sum(y_val[0]))
                # print(val_predictions)
                micro_f1 = f1_score(y_val, val_predictions, average='micro', zero_division=0)
                macro_f1 = f1_score(y_val, val_predictions, average='macro',zero_division=0)
                ans.append([macro_f1,i,j,k,val_predictions])
ans.sort(reverse=True)
for i in range(0,3):
    print(np.sum(ans[i][4]))
    print("F1 Score: " + str(ans[i][0]) + " Max Depth: " + str(ans[i][1]) + " Max Features: " + str(ans[i][2]) + " Criterion: " + str(ans[i][3]))
# val_predictions = mlb.inverse_transform(val_predictions)
# y_val = mlb.inverse_transform(y_val)

# Initialize an empty dictionary to store confusion matrices for each label
# confusion_matrices = {}

# # Compute confusion matrix for each label
# for label_idx, label in enumerate(mlb.classes_):
#     y_val_label = [1 if label in labels else 0 for labels in y_val]
#     val_predictions_label = [1 if label in labels else 0 for labels in val_predictions]
#     cm = confusion_matrix(y_val_label, val_predictions_label)
#     confusion_matrices[label] = cm

# # Print confusion matrices for each label
# for label, cm in confusion_matrices.items():
#     print(f'Confusion Matrix for Label "{label}":')
#     print(cm)